In [ ]:
!pip install app-store-scraper google-play-scraper bs4

# App Store Comment Scrap

In [ ]:
from app_store_scraper import AppStore
import pandas as pd
import numpy as np
import json

In [ ]:
def get_reviews(country, app_name, app_id):
    app = AppStore(
        country, app_name, app_id
    )
    app.review()
    return app.reviews


def export_report(data):
    df = pd.DataFrame(np.array(data),columns=['review'])
    df2 = df.join(pd.DataFrame(df.pop('review').tolist()))
    return df2

def app_store_scrap(queries):
    for i in range(len(queries)):
        country = queries['country'][i]
        app_name = queries['app_name'][i]
        app_id = queries['app_id'][i]
        app = get_reviews(country=country, app_name=app_name, app_id=app_id)
        df = export_report(app)
        df.to_csv(f'{app_name}.csv')
        print(f'{app_name}.csv saved')

In [ ]:
country = ['hk','hk','hk','hk','hk']
app_name = ['hang-seng-invest-express','hang-seng-personal-banking','hsbc-hk-mobile-banking','bochk','dbs-digibank-hk']
app_id = [1489143798,1039256353,1164066737,1534534188,1341250080]
queries = pd.DataFrame({'country':country,'app_name':app_name,'app_id':app_id})
app_store_scrap(queries)

# Google Play Comment Scrap

In [ ]:
from google_play_scraper import Sort, reviews_all

In [ ]:
def google_play_scrap(queries):
    for i in range(len(queries)):
        app_id = queries['app_id'][i]
        result = reviews_all(
            app_id,
            sleep_milliseconds=0, # defaults to 0
            lang='en', # defaults to 'en'
            country='hk', # defaults to 'us'
            sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
            filter_score_with=None # defaults to None(means all score)
        )
        pd.DataFrame(result).to_csv(f'{app_id}.csv')
        print(f'{app_id}.csv saved')

In [ ]:
app_id = ['com.hangseng.invex','com.hangseng.rbmobile','com.bochk.app.aos','hk.com.hsbc.hsbchkmobilebanking','com.dbs.hk.iwealthhk']
queries = pd.DataFrame({'app_id':app_id})
google_play_scrap(queries)

# HK Discuss Forum Comment Scrap

cloudflare protection enforced, this version is not applicable now.

In [ ]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import pandas as pd
import openpyxl

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

def extract(text):
    temp = text.find('發表')
    if temp == -1:
        return text[text.find(':', 4) + 9:]
    else:
        return text[text.find(' ', temp + 3):].strip()

def scrap(keywords):
    root = 'https://www.discuss.com.hk'

    for keyword in keywords:
        website = f'{root}/search.php?searchsubmit=true&srchtxt={keyword}&tr_h=rcju99Xx6eC&srchfid[]=57'

        # initialization
        links = []
        check = []
        result = requests.get(website)
        content = result.text
        soup = BeautifulSoup(content, "lxml")

        # pagination
        pagination = soup.find('div', class_='pagination')
        pages = pagination.find_all('a')
        try:
            last_page = int(pages[-2].text)
        except:
            last_page = 50

        # parse
        record = pd.DataFrame(columns=['date', 'comments', 'quote', 'link'])

        for page in tqdm(range(1, last_page + 1)):
            sub_result = requests.get(f'{website}&page={page}')
            sub_content = sub_result.text
            soup = BeautifulSoup(sub_content, 'lxml')
            box = soup.find('div', class_='search-result')
            for item in box.find_all('span', class_='search-result-subject'):
                link = item.find('a', href=True)
                links.append(link['href'])

        for link in tqdm(links):
            try:
                sub_result = requests.get(link)
                sub_content = sub_result.text
                soup = BeautifulSoup(sub_content, 'lxml')
                box = soup.find('div', class_='mainbox-container mb-t_msgfont')
                comments = box.find_all('div', class_='postmessage-content t_msgfont')
                dates = box.find_all('div', class_='post-date')
                temp1, temp2, temp3, temp4 = [],[],[],[]
                for i in range(len(comments)):
                    try:
                        date = extract(dates[i].get_text())
                        quote = comments[i].find('blockquote').get_text()
                        comment = comments[i].get_text()[len(quote) + 5:]
                        temp1.append(date)
                        temp2.append(comment)
                        temp3.append(quote)
                        temp4.append(link)
                    except:
                        date = extract(dates[i].get_text())
                        quote = ''
                        comment = comments[i].get_text()
                        temp1.append(date)
                        temp2.append(comment)
                        temp3.append(quote)
                        temp4.append(link)
            except:
                print('----- Link not working -----')
                print(link)
                check.append(link)
            page_data = pd.DataFrame({'date': temp1, 'comments': temp2, 'quote': temp3, 'link': temp4})
            record = pd.concat([page_data, record]).reset_index(drop=True)
        record['product'] = keyword       
        record.to_csv(f'{keyword}.csv',index=False)
        print(f'{keyword}.csv saved')


In [ ]:
keywords = ['股票掛鈎投資','可贖回']
scrap(keywords)